<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2025_01_10_Configure_Email_Filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest emails from a user's Microsoft email account, and filter out Sent Items, etc.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

To access your Microsoft email, assign these properties as Colab secrets: MICROSOFT_CLIENT_ID, MICROSOFT_CLIENT_SECRET and MICROSOFT_REFRESH_TOKEN.

---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.9/244.9 kB 9.5 MB/s eta 0:00:00


Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

In [3]:
os.environ['MICROSOFT_CLIENT_ID'] = userdata.get('MICROSOFT_CLIENT_ID')
os.environ['MICROSOFT_CLIENT_SECRET'] = userdata.get('MICROSOFT_CLIENT_SECRET')
os.environ['MICROSOFT_REFRESH_TOKEN'] = userdata.get('MICROSOFT_REFRESH_TOKEN')

Define Graphlit helper functions

In [21]:
from typing import List, Optional
import json

async def create_feed():
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Microsoft Email",
        type=enums.FeedTypes.EMAIL,
        email=input_types.EmailFeedPropertiesInput(
            type=enums.FeedServiceTypes.MICROSOFT_EMAIL,
            microsoft=input_types.MicrosoftEmailFeedPropertiesInput(
                type=enums.EmailListingTypes.PAST,
                refreshToken=os.environ['MICROSOFT_REFRESH_TOKEN'],
                clientId=os.environ['MICROSOFT_CLIENT_ID'],
                clientSecret=os.environ['MICROSOFT_CLIENT_SECRET'],
                includeDeletedItems=True,
                excludeSentItems=True,
                includeSpam=True,
                #inboxOnly=True,
            ),
            readLimit=25 # limiting to 25 emails
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def query_contents(search_text: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                search=search_text,
                searchType=enums.SearchTypes.HYBRID,
                types=[enums.ContentTypes.EMAIL]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [22]:
from IPython.display import display, Markdown
import time

# Remove any existing feeds; only needed for notebook example
await delete_all_feeds()

print('Deleted all feeds.')

Deleted all feeds.


Ingest Microsoft emails

In [23]:
feed_id = await create_feed()

if feed_id is not None:
    print(f'Created feed [{feed_id}].')

    # Wait for feed to complete, since ingestion happens asychronously
    done = False
    time.sleep(5)
    while not done:
        done = await is_feed_done(feed_id)

        if not done:
            time.sleep(10)

    print(f'Completed feed [{feed_id}].')

Created feed [2edb46eb-5967-47d0-bbbe-ea63974de967].
Completed feed [2edb46eb-5967-47d0-bbbe-ea63974de967].


Search within ingested emails

In [24]:
# NOTE: specify the text to search for in the filtered emails
search_text = None # return all email

# Query contents by search text
contents = await query_contents(search_text)

if contents is not None and len(contents) > 0:
    for content in contents:
        if content is not None:
            display(Markdown(f'### Found Microsoft email [{content.id}] that referenced search text [{search_text}].'))

            if content.original_date is not None:
                print("Date: " + content.original_date)

            metadata = content.email

            if metadata is not None:
                if metadata.subject is not None:
                    print("Subject: " + metadata.subject)
                if metadata.to is not None and len(metadata.to) > 0:
                    print("To: " + ", ".join([f'"{item.name}" <{item.email}>' for item in metadata.to if item is not None]))
                if metadata.from_ is not None and len(metadata.from_) > 0:
                    print("From: " + ", ".join([f'"{item.name}" <{item.email}>' for item in metadata.from_ if item is not None]))

            # NOTE: uncomment to see email markdown
            #display(Markdown(content.markdown))
            print()
        else:
            print('No content found')
else:
    print(f'No contents found by search [{search_text}].')


### Found Microsoft email [e470ddd2-c4e5-48ec-ba56-2af116881183] that referenced search text [None].

Date: 2025-01-11T05:02:32.000Z
Subject: [NuGet Gallery] Package published - Graphlit 2025.1.10.1
To: "Kirk Marple" <kirk@unstrukdata.onmicrosoft.com>
From: "NuGet Gallery" <support@nuget.org>



### Found Microsoft email [7f061ff2-4a15-4f97-8a17-039bc085b14c] that referenced search text [None].

Date: 2025-01-09T04:29:31.000Z
Subject: [NuGet Gallery] Package published - Graphlit 2025.1.8.1
To: "Kirk Marple" <kirk@unstrukdata.onmicrosoft.com>
From: "NuGet Gallery" <support@nuget.org>



### Found Microsoft email [a98c3101-6d9d-4c74-bc9a-442ac9bbc21b] that referenced search text [None].

...

Date: 2024-12-10T21:43:29.000Z
Subject: [NuGet Gallery] Package published - Graphlit 2024.12.10.1
To: "Kirk Marple" <kirk@unstrukdata.onmicrosoft.com>
From: "NuGet Gallery" <support@nuget.org>



### Found Microsoft email [f4840bb1-b06e-46bc-b611-db9550c2c594] that referenced search text [None].

Date: 2024-12-10T16:28:47.000Z
Subject: Remember to activate your benefits in Partner Center
To: "Kirk Marple" <kirk@unstrukdata.onmicrosoft.com>
From: "Microsoft ISV Success" <ISVSuccess@mails.microsoft.com>



### Found Microsoft email [f2c97ce8-e500-4dea-8bbc-a29c17292c09] that referenced search text [None].

Date: 2024-12-10T07:21:23.000Z
Subject: [NuGet Gallery] Package published - Graphlit 2024.12.9.1
To: "Kirk Marple" <kirk@unstrukdata.onmicrosoft.com>
From: "NuGet Gallery" <support@nuget.org>

